# Generates a list of clockwise-oriented neighbors of each precinct. 
Note: this may NOT properly handle cases where three or more precincts intersect at a point on the boundary of the state. In such cases, you should manually edit the output file. You can check for discrepancies with the cell at the bottom. 


In [24]:
input_file = "../test_graphs/MS_pct20.geojson"
output_file = "../test_graphs/MS_pct20_oriented_nbrs.json"

import JSON
geo = JSON.parsefile(input_file)

# remove duplicate points on the polygons 
for pct in geo["features"]
    poly = pct["geometry"]["coordinates"][1][1]
    i = 1 
    while i <= length(poly)
        if poly[i] == poly[i == length(poly) ? 1 : i+1]
            deleteat!(poly, i)
        else 
            i += 1 
        end 
    end
end
4

4

In [25]:
for s in  collect(keys(geo["features"][1]["properties"]))
    if s[1] !== 'G'
        println(s)
    end
end

COUNTY
POP20
PCTID


In [26]:
# sets neighbors of each polygon-point 
d = Dict{Vector{Float64}, Vector{Int64}}()
for i in eachindex(geo["features"])
    pct = geo["features"][i]
    nm = pct["properties"]["COUNTY"] * "," * pct["properties"]["PCTID"] 
    for border_pt in pct["geometry"]["coordinates"][1][1]
        if !haskey(d, border_pt)
            d[border_pt] = [i]
        else 
            push!(d[border_pt], i)
        end 
    end
end 

# sanity check: how many pcts touch the average point on a polygon? Should be around 2 
@show sum([length(v) for (k,v) in d]) / length(d)

sum([length(v) for (k, v) = d]) / length(d) = 1.9698424012679179


1.9698424012679179

## if you want no duplicates:  
(i.e., choose one edge to exist for pcts that touch each other in multiple spots. The de-duplication is consistent, i.e. a dupe edge from p1 to p2 will always remove the same one in order.)

This can cause issues with planarity if you use perfect matching... and anti-issues (maybe) with having dupes if you don't (maybe) 

# In general you probably *do* want duplicates; skip to below. 

In [6]:
oriented_nbrs = Dict{Int, Vector{Int}}() 
ngaps = 0 
for pct_idx in eachindex(geo["features"])
    pct = geo["features"][pct_idx]
    oriented_nbrs[pct_idx] = [] 
    poly = pct["geometry"]["coordinates"][1]

    spans = Dict{Int, Vector{Int}}()
    for j in eachindex(poly)
        border_pt = poly[j]
        next_border_pt = poly[j == length(poly) ? 1 : j+1]
        nbridx = 0 
        if length(d[border_pt]) == 1
            # maybe we want to add -1 here to denote the border of the state 
            if haskey(spans, -1)
                push!(spans[-1], j)
            else 
                spans[-1] = [j] 
            end 
        elseif length(d[border_pt]) >= 4
            continue 
        else 
            for other_pct in d[border_pt]
                if other_pct == pct_idx
                    continue 
                end 
                if haskey(spans, other_pct)
                    push!(spans[other_pct], j)
                else 
                    spans[other_pct] = [j] 
                end 
            end
        end
    end

    # if any of the spans are non-contiguous  .... 
    span_mids = Dict{Int, Float32}()
    for (other_pct, span) in spans 
        #if length(span) == 1 
        span_boundaries = [span[1]]
        for i = 1:length(span)-1
            if span[i+1] == span[i]+1
                continue 
            else 
                push!(span_boundaries, span[i])
                push!(span_boundaries, span[i+1])
            end
        end
        push!(span_boundaries, span[end])

        if length(span_boundaries) == 2 
            span_mids[other_pct] = (span_boundaries[1] + span_boundaries[2]) / 2
            continue
        elseif span_boundaries[1] == 1 && span_boundaries[end] == length(poly) && length(span_boundaries) == 4
            mid_span = (span_boundaries[3] + length(poly) + span_boundaries[2]) / 2
            span_mids[other_pct] = mid_span > length(poly) ? mid_span - length(poly) : mid_span
            continue 
            # all good; only one span 
        else 
            @show span_boundaries, pct_idx 
            if other_pct == -1 
                continue 
            end

            other_pct_geo = geo["features"][other_pct]["geometry"]["coordinates"][1]
            if span_boundaries[1] == 1 && span_boundaries[end] == length(poly)
                # take the boundary span, and delete the others 
                for i in 3:2:length(span_boundaries)-2
                    span_start, span_end = span_boundaries[i], span_boundaries[i+1]
                    for j = span_start:span_end 
                        deleteat!(other_pct_geo, 
                            findall(x->x == poly[j], other_pct_geo))
                    end
                end
                mid_span = (span_boundaries[end-1] + length(poly) + span_boundaries[2]) / 2
                span_mids[other_pct] = mid_span > length(poly) ? mid_span - length(poly) : mid_span
                #@show pct_idx 
                #@show other_pct 
                #@assert false 
            else 
                # take the first span, and delete the others from the other pct's geogrpahy 
                for i in 3:2:length(span_boundaries)
                    span_start, span_end = span_boundaries[i], span_boundaries[i+1]
                    for j = span_start:span_end 
                        deleteat!(other_pct_geo, 
                        findall(x->x == poly[j], other_pct_geo))
                    end
                end
                span_mids[other_pct] = (span_boundaries[1] + span_boundaries[2]) / 2
            end
        end
    end
    # now sort the spans (increasing order of the span start) 
    oriented_nbrs[pct_idx] = sort(collect(keys(span_mids)), by=(k->span_mids[k]))
end 

KeyError: KeyError: key Any[Any[-88.824801, 33.943156], Any[-88.82453, 33.943186], Any[-88.823227, 33.943193], Any[-88.822406, 33.943188], Any[-88.821536, 33.943184], Any[-88.821366, 33.943183], Any[-88.818841, 33.943173], Any[-88.816162, 33.943156], Any[-88.815217, 33.943148], Any[-88.813325, 33.943132], Any[-88.812821, 33.943122], Any[-88.812503, 33.94311], Any[-88.812406, 33.943107], Any[-88.811162, 33.943073], Any[-88.810918, 33.943067], Any[-88.810748, 33.943066], Any[-88.810074, 33.94306], Any[-88.808886, 33.943052], Any[-88.808053, 33.943043], Any[-88.807434, 33.943037], Any[-88.80738, 33.943033], Any[-88.807326, 33.943035], Any[-88.806668, 33.943028], Any[-88.805778, 33.943031], Any[-88.804873, 33.943035], Any[-88.803232, 33.943028], Any[-88.800973, 33.943028], Any[-88.800425, 33.943029], Any[-88.799372, 33.943021], Any[-88.797726, 33.943007], Any[-88.796583, 33.94299], Any[-88.795585, 33.942977], Any[-88.795247, 33.942972], Any[-88.793432, 33.942965], Any[-88.791542, 33.942966

# if you do want duplicates (multi-edges)

In [27]:
oriented_nbrs = Dict{Int, Vector{Int}}() 
ngaps = 0 
for pct_idx in eachindex(geo["features"])
    pct = geo["features"][pct_idx]
    oriented_nbrs[pct_idx] = [] 
    poly = pct["geometry"]["coordinates"][1][1]
    

    spans = Dict{Int, Vector{Int}}()
    for j in eachindex(poly)
        border_pt = poly[j]
        next_border_pt = poly[j == length(poly) ? 1 : j+1]
        nbridx = 0 
        if length(d[border_pt]) == 1 
            if haskey(spans, -1)
                push!(spans[-1], j)
            else 
                spans[-1] = [j] 
            end 
        elseif length(d[border_pt]) >= 4
            #@show pct["properties"]["NAME"]
            continue 
        else 
            for other_pct in d[border_pt]
                if other_pct == pct_idx
                    continue 
                end 
                if haskey(spans, other_pct)
                    push!(spans[other_pct], j)
                else 
                    spans[other_pct] = [j] 
                end 
            end
        end
    end

    # if any of the spans are non-contiguous  .... 
    span_mids = Dict{Float32, Int}()
    for (other_pct, span) in spans 
        #if length(span) == 1 
        span_boundaries = [span[1]]
        for i = 1:length(span)-1
            if span[i+1] == span[i]+1
                continue 
            else 
                push!(span_boundaries, span[i])
                push!(span_boundaries, span[i+1])
            end
        end
        push!(span_boundaries, span[end])

        if length(span_boundaries) == 2 
            span_mids[(span_boundaries[1] + span_boundaries[2]) / 2] = other_pct
            continue
        elseif span_boundaries[1] == 1 && span_boundaries[end] == length(poly) && length(span_boundaries) == 4
            mid_span = (span_boundaries[3] + length(poly) + span_boundaries[2]) / 2
            span_mids[mid_span > length(poly) ? mid_span - length(poly) : mid_span] = other_pct
            continue 
            # all good; only one span 
        else 
            @show span_boundaries, pct_idx 
            if span_boundaries[1] == 1 && span_boundaries[end] == length(poly)
                # has a boundary span. first, the non-boundary spans:  
                for i in 3:2:length(span_boundaries)-2
                    span_start, span_end = span_boundaries[i], span_boundaries[i+1]
                    mid_span = (span_start + span_end)/2
                    span_mids[mid_span] = other_pct
                end
                # boundary span:
                mid_span = (span_boundaries[end-1] + length(poly) + span_boundaries[2]) / 2
                span_mids[mid_span > length(poly) ? mid_span - length(poly) : mid_span] = other_pct
            else 
                # no boundary spans. add each span. 
                for i in 1:2:length(span_boundaries)
                    span_start, span_end = span_boundaries[i], span_boundaries[i+1]
                    span_mids[(span_start + span_end)/2] = other_pct
                end
            end
        end
    end
    # now sort the spans (increasing order of the span mid) 
    oriented_nbrs[pct_idx] = [span_mids[x] for x in sort(collect(keys(span_mids)))]
end 

(span_boundaries, pct_idx) = ([1, 7, 340, 345, 439, 466], 213)
(span_boundaries, pct_idx) = ([163, 197, 291, 296], 218)


(span_boundaries, pct_idx) = ([1901, 2293, 3806, 4383], 369)
(span_boundaries, pct_idx) = ([1222, 1799, 2124, 2516], 371)


(span_boundaries, pct_idx) = ([166, 178, 234, 248], 568)
(span_boundaries, pct_idx) = ([128, 142, 161, 173], 569)
(span_boundaries, pct_idx) = ([1225, 1627, 1632, 2069], 577)
(span_boundaries, pct_idx) = 

([330, 767, 1470, 1872], 580)


(span_boundaries, pct_idx) = ([421, 691, 746, 778], 803)
(span_boundaries, pct_idx) = ([28, 60, 428, 698], 809)


(span_boundaries, pct_idx) = ([742, 891, 1147, 1181], 837)
(span_boundaries, pct_idx) = ([134, 168, 214, 363], 853)


(span_boundaries, pct_idx) = ([13, 35, 317, 323], 1022)
(span_boundaries, pct_idx) = ([23, 45, 111, 117], 1029)


(span_boundaries, pct_idx) = ([82, 123, 350, 454], 1116)
(span_boundaries, pct_idx) = ([302, 406, 482, 523], 1122)


(span_boundaries, pct_idx) = ([113, 182, 184, 225], 1377)
(span_boundaries, pct_idx) = ([1, 26, 691, 732, 773, 816], 1380)
(span_boundaries, pct_idx) = ([164, 186, 206, 344], 1381)
(span_boundaries, pct_idx) = ([1, 14, 365, 503, 507, 515], 1383)


(span_boundaries, pct_idx) = ([60, 69, 73, 130], 1453)
(span_boundaries, pct_idx) = ([403, 460, 528, 537], 1459)


In [28]:
open(output_file,"w") do f
    JSON.print(f, oriented_nbrs)
end

# check the output 
any node pairs printed out likely intersect at a point at the border of the state. should check and manually add or remove them from the oriented neighbor list file. 

In [29]:
oriented_nbrs = JSON.parsefile(output_file)
oriented_nbrs = Dict{Int, Vector{Int}}([parse(Int, k) => v for (k,v) in oriented_nbrs])
for (k,v) in oriented_nbrs 
    for x in v 
        if x == -1 continue end 
        if !(k in oriented_nbrs[x])
            pct = geo["features"][k]
            nm1 = pct["properties"]["COUNTY"] * "," * pct["properties"]["NAME"] 

            pct = geo["features"][x]
            nm2 = pct["properties"]["COUNTY"] * "," * pct["properties"]["NAME"] 
        
            println("node ", k, " (", nm1, ") has node ", x, " (", nm2,"), in its oriented neighbor list, but not vice versa")
        end
    end
end

In [21]:
# you can, e.g., remove mismatches like this: (if 2514 has 2297 as a nbr but not vice versa)
filter!(x -> x != 21, oriented_nbrs[28])
filter!(x -> x != 28, oriented_nbrs[21])
open(output_file,"w") do f
    JSON.print(f, oriented_nbrs)
end

In [13]:
output_file

"../test_graphs/NC_pct21_oriented_nbrs.json"

In [30]:
@show oriented_nbrs[28]
oriented_nbrs[28] = [60, 318, 66, 21, 57, 21, 76, 29, 84, 34, 79, 59, 50, 42, 72, 58]
@show oriented_nbrs[21]
oriented_nbrs[21] = [411, -1, 38, 61, 82, 35, 87, 73, 68, 73, 26, 76, 28, 57, 28, 66, 318, 366];
open(output_file,"w") do f
    JSON.print(f, oriented_nbrs)
end

oriented_nbrs[28] = [60, 318, 66, 21, 57, 21, 76, 29, 84, 34, 79, 59, 50, 42, 72, 58]
oriented_nbrs[21] = 

[411, -1, 38, 61, 82, 35, 87, 73, 68, 73, 26, 76, 28, 57, 28, 66, 318, 366]
